In [1]:
import sys
sys.path.append("/media/HDD_2TB/marc/Video_Description/egocentric-video-description")
sys.path.append("/media/HDD_2TB/marc/multimodal_keras_wrapper")

from viddesc_model import VideoDesc_Model
from data_engine.prepare_data import build_dataset
from config import load_parameters

from keras_wrapper.cnn_model import loadModel, saveModel
from keras_wrapper.dataset import loadDataset
from keras_wrapper.extra.evaluation import *
from keras_wrapper.extra.callbacks import EarlyStopping

# Plot libraries
from matplotlib.patches import Rectangle
import matplotlib as mpl
mpl.use('Agg') # run matplotlib without X server (GUI)
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from PIL import Image as pilimage
from scipy import ndimage

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)


In [2]:
# Load basic dataset
params = load_parameters()
params['REBUILD_DATASET'] = False
dataset = build_dataset(params)
params['OUTPUT_VOCABULARY_SIZE'] = dataset.vocabulary_len[params['OUTPUTS_IDS_DATASET'][0]]

params['RELOAD'] = 2
params['STORE_PATH'] = 'trained_models/EDUB-SegDesc_features_ArcticVideoCaptionWithInit_txtemb_420_imgemb__lstmenc_600_lstm_484_deepout_maxout_Adam'

description
prev_description



In [3]:
# Load initial model
model = loadModel('/media/HDD_2TB/marc/Video_Description/egocentric-video-description/'+params['STORE_PATH'], 
                      params['RELOAD'])

/home/lifelogging/code/keras_marc/keras/layers/core.py:656: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, None, 301)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [4]:
video_model = VideoDesc_Model(params,
                                      type=params['MODEL_TYPE'],
                                      verbose=params['VERBOSE'],
                                      model_name=params['MODEL_NAME'],
                                      vocabularies=dataset.vocabulary,
                                      store_path=params['STORE_PATH'])

-----------------------------------------------------------------------------------
		VideoDesc_Model instance
-----------------------------------------------------------------------------------
_model_type: TemporallyLinkedVideoDescriptionAtt
name: EDUB-SegDesc_features-linked_TemporallyLinkedVideoDescriptionAtt_txtemb_301_imgemb__lstmenc_717_lstm_484_deepout_maxout_Adam
model_path: trained_models/EDUB-SegDesc_features_ArcticVideoCaptionWithInit_txtemb_420_imgemb__lstmenc_600_lstm_484_deepout_maxout_Adam
verbose: 1

MODEL params:
{'SAMPLE_ON_SETS': ['train', 'val'], 'MAX_OUTPUT_TEXT_LEN': 30, 'SAMPLING_SAVE_MODE': 'list', 'TRG_LAN': 'en', 'SAMPLING': 'max_likelihood', 'SAMPLE_EACH_UPDATES': 1, 'CLIP_C': 1.0, 'TRG_PRETRAINED_VECTORS_TRAINABLE': True, 'USE_PRELU': False, 'ENCODER_HIDDEN_SIZE': 717, 'REBUILD_DATASET': False, 'BATCH_NORMALIZATION_MODE': 1, 'OPTIMIZER': 'Adam', 'EVAL_EACH_EPOCHS': False, 'EPOCHS_FOR_SAVE': 1, 'USE_BATCH_NORMALIZATION': True, 'BATCH_SIZE': 64, 'MODEL_NAME':

In [5]:
old_model = loadModel(params['PRE_TRAINED_MODEL_STORE_PATH'], params['RELOAD'])

/home/lifelogging/code/keras_marc/keras/layers/core.py:656: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, None, 301)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [5]:
print old_model.model_next
print old_model.model_init

None
None


In [13]:
old_names = [layer.name for layer in old_model.model.layers]
new_names = [layer.name for layer in video_model.model.layers]

In [14]:
print old_names
print
print new_names

[u'video', u'bidirectional_encoder', u'merge_1', u'lambda_mean', u'dropout_1', u'state_below', u'initial_state', u'initial_memory', u'target_word_embedding', u'dropout_2', u'dropout_3', u'attlstmcond_1', u'dropout_4', u'logit_ctx', u'logit_lstm', u'lambda_1', u'additional_input', u'activation_1', u'dropout_5', u'description']

['video', 'bidirectional_encoder_LSTM', 'merge_2', 'input_video_gaussian_noise', 'input_video_batch_normalization', 'prev_description', 'target_word_embedding', 'state_below', 'lambda_mean', 'encoder_prev_descLSTM', 'initial_state', 'initial_memory', 'target_word_embedding_gaussian_noise', 'prev_desc_enc_gaussian_noise', 'initial_state_gaussian_noise', 'initial_memory_gaussian_noise', 'target_word_embedding_batch_normalization', 'prev_desc_enc_batch_normalization', 'initial_state_batch_normalization', 'initial_memory_batch_normalization', 'decoder_AttLSTMCond2Inputs', 'proj_h0_gaussian_noise', 'proj_h0_batch_normalization', 'logit_ctx', 'logit_prev', 'logit_lstm'

In [7]:
old_layer_dict = dict([(layer.name, layer) for layer in old_model.model.layers])
new_layer_dict = dict([(layer.name, layer) for layer in video_model.model.layers])

In [16]:
old_weights = old_layer_dict['bidirectional_encoder'].get_weights()
for w in old_weights:
    print w.shape
print
new_weights = new_layer_dict['bidirectional_encoder_LSTM'].get_weights()
for w in new_weights:
    print w.shape

(1024, 2868)
(717, 2868)
(2868,)
(1024, 2868)
(717, 2868)
(2868,)

(1024, 2868)
(717, 2868)
(2868,)
(1024, 2868)
(717, 2868)
(2868,)


In [17]:
old_weights = old_layer_dict['initial_state'].get_weights()
for w in old_weights:
    print w.shape
print
new_weights = new_layer_dict['initial_state'].get_weights()
for w in new_weights:
    print w.shape

(2458, 484)
(484,)

(2458, 484)
(484,)


In [18]:
old_weights = old_layer_dict['target_word_embedding'].get_weights()
for w in old_weights:
    print w.shape
print
new_weights = new_layer_dict['target_word_embedding'].get_weights()
for w in new_weights:
    print w.shape

(13578, 301)

(832, 301)


In [19]:
old_weights = old_layer_dict['attlstmcond_1'].get_weights()
for w in old_weights:
    print w.shape
print
new_weights = new_layer_dict['decoder_AttLSTMCond2Inputs'].get_weights()
for w in new_weights:
    print w.shape

(2458,)
(484, 2458)
(2458, 2458)
(2458,)
(26,)
(301, 1936)
(2458, 1936)
(484, 1936)
(1936,)

(2458,)
(484, 2458)
(2458, 2458)
(2458,)
()
(2458, 1936)
(484, 1936)
(484, 1936)
(301, 1936)
(1936,)
(484,)
(484, 484)
(484, 484)
(484,)
()


In [20]:
old_weights = old_layer_dict['logit_ctx'].get_weights()
for w in old_weights:
    print w.shape
print
new_weights = new_layer_dict['logit_ctx'].get_weights()
for w in new_weights:
    print w.shape

(2458, 301)
(301,)

(2458, 301)
(301,)


In [21]:
old_weights = old_layer_dict['logit_lstm'].get_weights()
for w in old_weights:
    print w.shape
print
new_weights = new_layer_dict['logit_lstm'].get_weights()
for w in new_weights:
    print w.shape

(484, 301)
(301,)

(484, 301)
(301,)


In [23]:
old_weights = old_layer_dict['description'].get_weights()
for w in old_weights:
    print w.shape
print
new_weights = new_layer_dict['description'].get_weights()
for w in new_weights:
    print w.shape

(301, 13578)
(13578,)

(150, 832)
(832,)


In [6]:
from keras_wrapper.cnn_model import transferWeights as tw

video_model = tw(old_model, video_model, params['LAYERS_MAPPING'])

In [11]:
print video_model.vocabularies.keys()
len(video_model.vocabularies['prev_description']['idx2words'].keys())

NameError: name 'video_model' is not defined

In [10]:
len(dataset.vocabulary['prev_description']['idx2words'].keys())

13578

In [4]:
dataset.vocabulary.keys()

['prev_description', 'state_below', 'description']

In [9]:
len(dataset.vocabulary['description']['idx2words'].keys())

13578

In [3]:
dataset_pretrained = loadDataset(params['DATASET_STORE_PATH']+'Dataset_'+params['PRE_TRAINED_DATASET_NAME']+'.pkl')

In [8]:
print dataset.vocabulary_len
print dataset_pretrained.vocabulary_len

{'prev_description': 13578, 'state_below': 13578, 'description': 13578}
{'description': 13578}


In [7]:
dataset.vocabulary['description'] = copy.deepcopy(dataset_pretrained.vocabulary['description'])
dataset.vocabulary_len['description'] = copy.deepcopy(dataset_pretrained.vocabulary_len['description'])

dataset.vocabulary['prev_description'] = copy.deepcopy(dataset_pretrained.vocabulary['description'])
dataset.vocabulary_len['prev_description'] = copy.deepcopy(dataset_pretrained.vocabulary_len['description'])

dataset.vocabulary['state_below'] = copy.deepcopy(dataset_pretrained.vocabulary['description'])
dataset.vocabulary_len['state_below'] = copy.deepcopy(dataset_pretrained.vocabulary_len['description'])

In [3]:
for id_old, id_new in params['VOCABULARIES_MAPPING'].iteritems():
            print id_old
            print id_new
            print

description
prev_description



In [4]:
params['VOCABULARIES_MAPPING']

{'description': 'prev_description'}